In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import string
import yaml
from rich import print, inspect
from subprocess import run as sub_run
from pyfig.pretty import print_table

from pathlib import Path

log = Path('.log')

from pydantic import BaseModel, Field

from uuid import uuid1

from typing import Optional, Any
from pydantic import BaseModel
import os
from rich.pretty import pprint
# from ts_diff_qc import add_model

from pyfig.plugins import Env
env: Env 				= Env(_env_file= '.env')

tips = """
```bash
docker container ls
docker container commit c16378f943fe new-image-name:latest

docker image ls
```
"""

Allowed dtypes:  typing.Literal['float64', 'float32']


In [3]:
class ResourceLimits(BaseModel):
	memory: str  = '10Gi' # Maximum amount of memory the container can use
	cpu: str  = '1' # Maximum number of CPU cores the container can use
	gpu: str = '0' # ""

class ResourceRequests(BaseModel):
	memory: str = '10Gi'  # Amount of memory the container requests
	cpu: str = '1'   # Number of CPU cores the container requests
	gpu: str = '0' # ""

class Volume(BaseModel):
	name: str = Field(default_factory= lambda: str(uuid1())) # Name of the volume
	persistentVolumeClaim: dict[str, str]  # Name of the PersistentVolumeClaim

class volumeMount(BaseModel):
	name: str = Field(default_factory= lambda: str(uuid1())) # Name of the volume
	mountPath: str  # Path to mount the volume inside the container

class Container(BaseModel):
	name: str = Field(default_factory= lambda: str(uuid1())) # Name of the container
	image: str = "gitlab-registry.nrp-nautilus.io/usra/scb-env-one:latest" # base docker
	command: list[str] = ["/bin/bash", "-c"] # run bash at start
	args: list[str] = [". activate base; sleep infinity"] # Arguments to the command
	volumeMounts: list[volumeMount]  # Mount points for volumes
	resources: dict[str, ResourceLimits | ResourceRequests]  # Resource limits for the container

class Metadata(BaseModel): 
	""" info about the job """
	name: str       = "ata-pod"
	namespace: str  = "scb-usra" 

class Spec(BaseModel):
	containers: list[Container]
	volumes: list[Volume]
	restartPolicy: str = "Never"

class Pod(BaseModel):
	apiVersion: str = "v1"  # Kubernetes API version
	kind: str       = "Pod" # Type of Kubernetes resource
	metadata: dict[str, str] = Metadata() # Metadata for the resource
	spec: Spec # Specification for the resource

vol1 = Volume(
	name= "scb-usra",  # name in the pod
	persistentVolumeClaim= dict(
		claimName= "scb-usra",  # ???
))

con1 = Container(
	name=       "a container",  # Name of the container
	volumeMounts= [
		volumeMount(name= "scb-usra", mountPath=  "/scb-usra"),
	],
	resources= dict(
		request= ResourceRequests(memory="10Gi", cpu= "1"),
		limits= ResourceLimits(memory="10Gi", cpu= "1"),
	)
)

In [4]:
pod = Pod(
    metadata= dict(
        name= "scb-usra",
        namespace= "scb-usra",
    ),
    spec= Spec(
        containers= [con1],
        volumes= [vol1],
        restartPolicy= "Never",
    )
)
inspect(pod)

╭─────────────────────────────────────── <class '__main__.Pod'> ───────────────────────────────────────╮
│ ╭──────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Pod(                                                                                             │ │
│ │ │   apiVersion='v1',                                                                             │ │
│ │ │   kind='Pod',                                                                                  │ │
│ │ │   metadata={'name': 'scb-usra', 'namespace': 'scb-usra'},                                      │ │
│ │ │   spec=Spec(                                                                                   │ │
│ │ │   │   containers=[                                                                             │ │
│ │ │   │   │   Container(                                                                           │ │
│ │ │   │   │   │   name='a container',                                                              │ │
│ │ │   │   │   │   image='gitlab-registry.nrp-nautilus.io/usra/scb-env-one:latest',                 │ │
│ │ │   │   │   │   command=['/bin/bash', '-c'],                                                     │ │
│ │ │   │   │   │   args=['. activate base; sleep infinity'],                                        │ │
│ │ │   │   │   │   volumeMounts=[volumeMount(name='scb-usra', mountPath='/scb-usra')],              │ │
│ │ │   │   │   │   resources={                                                                      │ │
│ │ │   │   │   │   │   'request': ResourceLimits(memory='10Gi', cpu='1', gpu='0'),                  │ │
│ │ │   │   │   │   │   'limits': ResourceLimits(memory='10Gi', cpu='1', gpu='0')                    │ │
│ │ │   │   │   │   }                                                                                │ │
│ │ │   │   │   )                                                                                    │ │
│ │ │   │   ],                                                                                       │ │
│ │ │   │   volumes=[Volume(name='scb-usra', persistentVolumeClaim={'claimName': 'scb-usra'})],      │ │
│ │ │   │   restartPolicy='Never'                                                                    │ │
│ │ │   )                                                                                            │ │
│ │ )                                                                                                │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                      │
│ apiVersion = 'v1'                                                                                    │
│       kind = 'Pod'                                                                                   │
│   metadata = {'name': 'scb-usra', 'namespace': 'scb-usra'}                                           │
│       spec = Spec(                                                                                   │
│                  containers=[                                                                        │
│                      Container(                                                                      │
│                          name='a container',                                                         │
│                          image='gitlab-registry.nrp-nautilus.io/usra/scb-env-one:latest',            │
│                          command=['/bin/bash', '-c'],                                                │
│                          args=['. activate base; sleep infinity'],                                   │
│                          volumeMounts=[volumeMount(name='scb-usra', mountPath='/scb-usra')],         │
│                          resources={                                                                 │
│         

In [8]:
import yaml
from pyfig.files import read
# pprint(pod)
# yaml.dump(pod.dict(), open('pod.yaml', 'w'))
# pod_yaml = read('pod.yaml')
# pprint(pod_yaml.splitlines())
# pprint(pod.schema())

In [9]:
from kub import Kubernetes, Pod, Container, Volume, volumeMount, ResourceLimits, ResourceRequests, Metadata, Spec

con1 = Container(name = 'a-container',
                 
    image = 'docker.io/xmkqv/simple:latest',

    command = ['/bin/bash', '-c'],
    args = ['echo "hello world"'],

    volumeMounts = [volumeMount(name= 'scb-usra', mountPath= '/scb-usra'),],
    
    resources = dict(
        requests = ResourceRequests(memory= '10Gi', cpu= '1'),
        limits = ResourceLimits(memory= '10Gi', cpu= '1'),
    ), 
)

vol1 = Volume(name = 'scb-usra',
    persistentVolumeClaim = dict(
        claimName = 'scb-usra',
    )
)

pod = Pod(
    metadata= Metadata(name= 'max-pod', namespace= 'scb-usra'),
    spec= Spec(
        containers= [con1],
        volumes= [vol1],
        restartPolicy= 'Never',


    ),
    apiVersion= "v1",
    kind= "Pod",
)

kub: Kubernetes = Kubernetes(
    pod             = pod,
    n_gpu 			= 0,
    n_cpu           = 1,
    container_name  = 'gpu-container',
    link_cmd        = 'ln -s /cephfs ~/cephfs',
    kub_image		= 'docker.io/xmkqv/simple:latest',
    repo			= 'github.com:xmax1/ts_diff_qc.git',
)


# kub.create_pod(pod= pod)
path = Path('tmp') / pod.metadata.name
path = path.with_suffix(suffix= '.yaml')
path.parent.mkdir(exist_ok= True, parents= True)
pprint(pod)
print(path)
yaml.dump(pod.dict(exclude_none= True), open(path, 'w'))

Pod(
│   apiVersion='v1',
│   kind='Pod',
│   metadata=Metadata(name='max-pod', namespace='scb-usra'),
│   spec=Spec(
│   │   containers=[
│   │   │   Container(
│   │   │   │   name='a-container',
│   │   │   │   image='docker.io/xmkqv/simple:latest',
│   │   │   │   command=['/bin/bash', '-c'],
│   │   │   │   args=['echo "hello world"'],
│   │   │   │   volumeMounts=[volumeMount(name='scb-usra', mountPath='/scb-usra')],
│   │   │   │   resources={
│   │   │   │   │   'requests': ResourceLimits(memory='10Gi', cpu='1', gpu=None, nvidia_gpu=None),
│   │   │   │   │   'limits': ResourceLimits(memory='10Gi', cpu='1', gpu=None, nvidia_gpu=None)
│   │   │   │   }
│   │   │   )
│   │   ],
│   │   volumes=[Volume(name='scb-usra', persistentVolumeClaim={'claimName': 'scb-usra'})],
│   │   restartPolicy='Never'
│   )
)

tmp/max-pod.yaml

In [13]:
from subprocess import run
# run(f'kubectl create -f {path}', shell= True, text= True)
# kub.get_pods()
_ = kub.delete_all_my_pods(tag= 'max-pod', show= True)

┏━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┓
┃ pod ┃ "max-pod" ┃ deleted ┃
┡━━━━━╇━━━━━━━━━━━╇━━━━━━━━━┩
└─────┴───────────┴─────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 87m   │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 4d23h │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 49m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h26m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

In [20]:
from kub import Kubernetes, Pod, Container, Volume, volumeMount, ResourceLimits, ResourceRequests, Metadata, Spec

con1 = Container(name = 'gpu-container',
                 
    # image = 'docker.io/xmkqv/simple:latest',
    image= 'gitlab-registry.nrp-nautilus.io/usra/scb-env-one:latest',

    command = ['/bin/bash', '-c'],
    args = ['pip install tqdm; sleep infinity'],

    volumeMounts = [volumeMount(name= 'scb-usra', mountPath= '/scb-usra'),],
    
    resources = dict(
        limits = ResourceLimits(nvidia_gpu = 1),
    ), 
)

vol1 = Volume(name = 'scb-usra',
    persistentVolumeClaim = dict(
        claimName = 'scb-usra',
    )
)

pod = Pod(
    metadata= Metadata(name= 'gpu-pod-xmkqv', namespace= 'scb-usra'),
    spec= Spec(
        containers= [con1],
        volumes= [vol1],
        restartPolicy= 'Never',
    ),
    apiVersion= "v1",
    kind= "Pod",
)

kub: Kubernetes = Kubernetes(
    namespace= 'scb-usra',
    link_cmd= 'ln -s /cephfs ~/cephfs',
    repo= 'github.com:xmax1/ts_diff_qc.git',
)

# metadata:
#   name: 
#   namespace: scb-usra
# spec:
#   containers:
#   - args:
#     - pip install tqdm; sleep infinity
#     command:
#     - /bin/bash
#     - -c
#     image: docker.io/xmkqv/simple:latest
#     name: gpu-container
#     resources:
#       limits:
#         nvidia.com/gpu: 1
#     volumeMounts:
#     - mountPath: /scb-usra
#       name: scb-usra
#   restartPolicy: Never
#   volumes:
#   - name: scb-usra
#     persistentVolumeClaim:
#       claimName: scb-usra


In [21]:
kub.get_pods()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 91m   │
│ gpu-pod-xmkqv                         │ 0/1   │ Completed         │ 0        │ 3m45s │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 4d23h │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 53m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h30m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

In [23]:
kub.delete_all_my_pods(tag= 'gpu-pod-xmkqv', show= True, )

┏━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ pod ┃ "gpu-pod-xmkqv" ┃ deleted ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
└─────┴─────────────────┴─────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 92m   │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 4d23h │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 54m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h31m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

In [24]:
kub.create_pod(pod= pod)
kub.get_pods(periodic= (5, 120))
# _ = kub.delete_all_my_pods(tag= 'max-pod', show= True)

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ pod/gpu-pod-xmkqv ┃ created ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
└───────────────────┴─────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 93m   │
│ gpu-pod-xmkqv                         │ 0/1   │ ContainerCreating │ 0        │ 1s    │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 5d    │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 55m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h33m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 93m   │
│ gpu-pod-xmkqv                         │ 0/1   │ ContainerCreating │ 0        │ 2s    │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 5d    │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 55m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h33m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

elapsed: 6.34 seconds

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 93m   │
│ gpu-pod-xmkqv                         │ 0/1   │ ContainerCreating │ 0        │ 9s    │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 5d    │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 55m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h33m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

...

elapsed: 6.30 seconds

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 98m   │
│ gpu-pod-xmkqv                         │ 1/1   │ Running           │ 0        │ 4m34s │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 5d    │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 60m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h37m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

elapsed: 6.30 seconds

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┓
┃ NAME                                  ┃ READY ┃ STATUS            ┃ RESTARTS ┃ AGE   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━┩
│ ata-pod                               │ 0/1   │ ContainerCreating │ 0        │ 98m   │
│ gpu-pod-xmkqv                         │ 1/1   │ Running           │ 0        │ 4m41s │
│ hub-859c877b85-fzfrj                  │ 1/1   │ Running           │ 0        │ 5d    │
│ jupyter-ataaakbari-40googlemail-2ecom │ 1/1   │ Running           │ 0        │ 60m   │
│ jupyter-fawudarski-40gmail-2ecom      │ 1/1   │ Running           │ 0        │ 7h37m │
│ proxy-5ddd4b47c9-xhb2d                │ 1/1   │ Running           │ 0        │ 43h   │
└───────────────────────────────────────┴───────┴───────────────────┴──────────┴───────┘

KeyboardInterrupt: 

In [ ]:
from pyfig.plugins import Logger, Paths, d_to_wb, Env, Opt, MULTIMODE
